In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from pathlib import Path

In [3]:
email = os.environ["USER_EMAIL"]
password = os.environ["USER_PASSWORD"]
URL = os.environ["URL"]

In [4]:
driver = webdriver.Chrome()

In [5]:
def login(url: str):
    driver.get(URL)
    time.sleep(10)
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    driver.find_element("name", "email").send_keys(email)
    driver.find_element("name", "password").send_keys(password)
    driver.find_elements("css selector", "button")[1].click()

    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))

In [6]:
login(URL)

In [7]:
nav = driver.find_element("css selector", "div.mobile-navigator")

nav.find_elements("css selector", "a")[3].click()

In [317]:
def reload():
#     nav = driver.find_element("css selector", "div.mobile-navigator")
#     nav.find_elements("css selector", "a")[3].click()
    matches_container = driver.find_element("css selector", ".match-wrapper")
    matches = matches_container.find_elements("css selector", ".match-card")
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    return matches

In [254]:
matches = reload()
len(matches)

10

In [229]:
[team.text for team in matches[0].find_elements("css selector", ".team-score")]

['2', '0']

In [10]:
def get_results(teams: list[str], scores: list[str]):
    if teams[0].lower() == "sporting lagos" and int(scores[0]) > int(scores[1]):
        result = "W"
        
    elif teams[1].lower() == "sporting lagos" and int(scores[1]) > int(scores[0]):
        result = "W"
        
    elif int(scores[0]) == int(scores[1]):
        result = "D"
        
    else:
        result = "L"
        
    return result

In [11]:
team_a = ["Enugu", "Sporting Lagos"]
score_a = ["0", "2"]
get_results(team_a, score_a)

'W'

In [237]:
matches_list = []

In [238]:
def get_data(match, matches_list):
    try:
        teams = [team.text for team in match.find_elements("css selector", ".team")]
        score_list = [score.text for score in match.find_elements("css selector", ".team-score")]
        home_away = "home" if teams[0].lower() == "sporting lagos" else "away"
        opponent = [team for team in teams if team.lower() != "sporting lagos"]
        scores = f"{score_list[0]} - {score_list[1]}"
        result = get_results(teams, score_list)

        match_data = pd.DataFrame()
        match_data["opponent"] = opponent
        match_data["home_away"] = home_away
        match_data["score"] = scores
        match_data["result"] = result

        if match.find_element("css selector", "button").get_attribute("disabled") == "true":
            matches_list.append(match_data)

        else:

            try:
                match.find_element("css selector", "button").click()

            except Exception:
                match.find_element("css selector", "button").click()

            time.sleep(5)
            try:
                stats_names, stats_values = technical_analysis()

            except Exception:
                stats_names, stats_values = technical_analysis()

            for i in range(len(stats_names)):
                match_data[stats_names[i]] = stats_values[i]

            matches_list.append(match_data)
            
        driver.back()
        driver.refresh()
        try: 
            matches = reload()
        except:
            matches = reload()
    except Exception as e:
        raise e


In [315]:
driver.refresh()

In [324]:
matches[5].text

'LEAGUE\n29 APRIL, 2023\nJoy Cometh\nSporting Lagos\n2\n2\nVIEW MATCH REPORT & HIGHLIGHTS'

In [328]:
get_data(matches[5], matches_list)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".match-wrapper"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62A622142+3514994]
	(No symbol) [0x00007FF62A240CE2]
	(No symbol) [0x00007FF62A0E76AA]
	(No symbol) [0x00007FF62A131860]
	(No symbol) [0x00007FF62A13197C]
	(No symbol) [0x00007FF62A174EE7]
	(No symbol) [0x00007FF62A15602F]
	(No symbol) [0x00007FF62A1728F6]
	(No symbol) [0x00007FF62A155D93]
	(No symbol) [0x00007FF62A124BDC]
	(No symbol) [0x00007FF62A125C64]
	GetHandleVerifier [0x00007FF62A64E16B+3695259]
	GetHandleVerifier [0x00007FF62A6A6737+4057191]
	GetHandleVerifier [0x00007FF62A69E4E3+4023827]
	GetHandleVerifier [0x00007FF62A3704F9+689705]
	(No symbol) [0x00007FF62A24C048]
	(No symbol) [0x00007FF62A248044]
	(No symbol) [0x00007FF62

In [327]:
matches = reload()
len(matches)

6

In [329]:
len(matches_list)

16

In [330]:
matches_list[15]

,opponent,home_away,score,result,Long Pass - Successful,Long Pass - Unsuccessful,Short Pass - Successful,Short Pass - Unsuccessful,Line Break - Successful,Line Break - Unsuccessful,...,Catches,Drop,Goal Conceded,Throw,Punch,Smother,Yellow Card,Red Card,Match Timeline,Pitch area
0,Joy Cometh,away,2 - 2,D,35,30,226,20,24,24,...,1,0,2,0,0,0,0,0,None,None


In [331]:
def technical_analysis():
    driver.find_element("id", "TECHNICAL-ANALYSIS-tab").click()
    WebDriverWait(driver=driver, timeout=10).until(
        lambda x: x.execute_script("return document.readyState === 'complete'"))
    
    stat_names = [stat_name.text for stat_name in driver.find_elements("css selector", "p.title")]
    stat_values = [stat_value.text for stat_value in driver.find_elements("css selector", "p.total")]
    stat_values.extend([None, None])
    
    return stat_names, stat_values

In [133]:
stat_name, stat_val = technical_analysis()

In [37]:
stat_name = driver.find_elements("css selector", "p.title")
stat_val =  driver.find_elements("css selector", "p.total")

In [134]:
print(stat_name[14])
stat_val[14]

Dribbles - Successful


'10'

In [322]:
driver.back()
driver.refresh()

In [337]:
pd.set_option("display.max_columns", None)

In [348]:
full_stats = pd.concat(matches_list).reset_index(drop=True)

In [349]:
full_stats

,opponent,home_away,score,result,Long Pass - Successful,Long Pass - Unsuccessful,Short Pass - Successful,Short Pass - Unsuccessful,Line Break - Successful,Line Break - Unsuccessful,Low Cross - Successful,Low Cross - Unsuccessful,High Cross - Successful,High Cross - Unsuccessful,PullBack Cross - Successful,PullBack Cross - Unsuccessful,Floated Cross - Successful,Floated Cross - Unsuccessful,Dribbles - Successful,Dribbles - Unsuccessful,Ball Progression,Possession Lost,Penalty Won,Foul Won,Offside,Recovery,Throw-In,Shield Ball Out Of Play,Goals,Outside Box - On Target,Outside Box - Off Target,Inside Box - On Target,Inside Box - Off Target,Shot Blocked,Assist,Key Pass,Freekick - On Target,Freekick - Off Target,Freekick - Blocked,Freekick - Short Pass - Successful,Freekick - Short Pass - Unsuccessful,Freekick - Long Pass - Successful,Freekick - Long Pass - Unsuccessful,Penalty Shot Scored,Penalty Shot Missed,Penalty Shot Saved,Penalty Shootout Scored,Penalty Shootout Missed,Penalty Shootout Saved,Penalty Shootout Goal Conceded,Corner Short Pass - Successful,Corner Short Pass - Unsuccessful,Corner Long Pass - Successful,Corner Long Pass - Unsuccessful,Corner Low Cross - Successful,Corner Low Cross - Unsuccessful,Corner High Cross - Successful,Corner High Cross - Unsuccessful,Interception,Blocks,Clearance,Foul Conceded,Penalty Conceded,Tackles Won,Tackles Lost,Offside Provoked,Aerial Duel Won,Aerial Duel Lost,Ground Duel Won,Ground Duel Lost,Own Goals,Error,Saves,Catches,Drop,Goal Conceded,Throw,Punch,Smother,Yellow Card,Red Card,Match Timeline,Pitch area
0,Enugu Rangers,away,2 - 0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lobi Stars,away,1 - 0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kano Pillars,home,3 - 0,W,112,44,220,18,14,14,0,1,0,0,0,0,0,0,10,8,6,14,0,12,3,1,13,0,3,3,3,4,2,1,2,5,0,0,0,6,0,5,1,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,27,3,27,22,0,16,9,3,5,9,4,2,0,0,4,2,0,0,5,0,0,2,0,None,None
3,Kwara United,away,2 - 1,L,39,61,90,11,6,6,0,0,0,1,0,0,0,0,6,1,0,33,0,8,2,9,25,0,1,2,2,2,0,0,1,2,0,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,13,2,67,21,1,15,7,1,17,9,7,3,0,2,4,2,0,2,2,0,0,6,0,None,None
4,Sunshine Stars,away,1 - 0,L,81,31,232,20,12,12,0,0,0,2,0,0,0,1,9,2,7,37,0,13,0,10,14,4,0,1,2,0,2,4,0,1,0,0,1,8,0,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,3,1,42,10,0,7,3,3,9,5,3,1,0,4,5,4,0,1,6,0,0,1,0,None,None
5,Niger Tornadoes,home,2 - 1,W,65,45,174,28,7,7,0,0,0,0,0,0,1,3,7,3,1,19,0,15,6,4,0,0,2,1,2,3,3,1,1,2,0,0,0,5,0,4,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,10,2,34,15,0,20,0,1,14,1,4,0,0,0,0,3,0,1,4,0,0,1,0,None,None
6,Remo Stars,away,2 - 1,L,48,36,78,9,5,5,0,0,1,2,0,0,0,1,12,9,3,32,0,18,0,5,21,0,1,2,1,2,5,4,1,4,0,1,0,3,0,9,5,0,0,0,0,0,0,0,3,0,0,0,0,0,1,3,13,0,23,19,1,11,3,2,15,13,5,3,0,1,3,6,0,2,3,0,0,1,0,None,None
7,Doma United,home,0 - 0,D,71,38,205,11,11,11,0,0,0,4,0,0,0,1,6,11,9,19,0,25,1,5,1,0,0,2,6,2,3,3,0,4,0,1,1,12,0,5,1,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,10,3,26,10,0,7,0,1,11,6,7,2,0,1,3,5,0,0,8,0,0,1,0,None,None
8,Akwa United,away,0 - 0,D,47,43,145,23,11,11,0,0,0,0,0,0,1,0,4,4,4,23,0,14,4,4,0,0,0,0,1,0,2,2,0,1,0,1,0,3,0,6,3,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,17,3,54,16,0,15,3,2,14,6,1,1,0,0,2,6,0,0,2,0,0,2,0,None,None
9,Gombe United,home,2 - 0,W,72,34,155,10,13,13,0,0,0,0,0,0,0,2,12,8,12,20,0,17,4,4,0,0,2,1,2,6,5,3,2,9,0,1,0,5,0,8,4,0,0,0,0,0,0,0,3,0,1,0,0,0,1,0,20,5,34,15,1,11,1,0,18,14,1,1,0,2,6,5,0,0,3,0,0,2,0,None,None


In [356]:
full_stats

,opponent,home_away,score,result,Long Pass - Successful,Long Pass - Unsuccessful,Short Pass - Successful,Short Pass - Unsuccessful,Line Break - Successful,Line Break - Unsuccessful,Low Cross - Successful,Low Cross - Unsuccessful,High Cross - Successful,High Cross - Unsuccessful,PullBack Cross - Successful,PullBack Cross - Unsuccessful,Floated Cross - Successful,Floated Cross - Unsuccessful,Dribbles - Successful,Dribbles - Unsuccessful,Ball Progression,Possession Lost,Penalty Won,Foul Won,Offside,Recovery,Throw-In,Shield Ball Out Of Play,Goals,Outside Box - On Target,Outside Box - Off Target,Inside Box - On Target,Inside Box - Off Target,Shot Blocked,Assist,Key Pass,Freekick - On Target,Freekick - Off Target,Freekick - Blocked,Freekick - Short Pass - Successful,Freekick - Short Pass - Unsuccessful,Freekick - Long Pass - Successful,Freekick - Long Pass - Unsuccessful,Penalty Shot Scored,Penalty Shot Missed,Penalty Shot Saved,Penalty Shootout Scored,Penalty Shootout Missed,Penalty Shootout Saved,Penalty Shootout Goal Conceded,Corner Short Pass - Successful,Corner Short Pass - Unsuccessful,Corner Long Pass - Successful,Corner Long Pass - Unsuccessful,Corner Low Cross - Successful,Corner Low Cross - Unsuccessful,Corner High Cross - Successful,Corner High Cross - Unsuccessful,Interception,Blocks,Clearance,Foul Conceded,Penalty Conceded,Tackles Won,Tackles Lost,Offside Provoked,Aerial Duel Won,Aerial Duel Lost,Ground Duel Won,Ground Duel Lost,Own Goals,Error,Saves,Catches,Drop,Goal Conceded,Throw,Punch,Smother,Yellow Card,Red Card
0,Kano Pillars,home,3 - 0,W,112,44,220,18,14,14,0,1,0,0,0,0,0,0,10,8,6,14,0,12,3,1,13,0,3,3,3,4,2,1,2,5,0,0,0,6,0,5,1,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,27,3,27,22,0,16,9,3,5,9,4,2,0,0,4,2,0,0,5,0,0,2,0
1,Kwara United,away,2 - 1,L,39,61,90,11,6,6,0,0,0,1,0,0,0,0,6,1,0,33,0,8,2,9,25,0,1,2,2,2,0,0,1,2,0,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,13,2,67,21,1,15,7,1,17,9,7,3,0,2,4,2,0,2,2,0,0,6,0
2,Sunshine Stars,away,1 - 0,L,81,31,232,20,12,12,0,0,0,2,0,0,0,1,9,2,7,37,0,13,0,10,14,4,0,1,2,0,2,4,0,1,0,0,1,8,0,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,3,1,42,10,0,7,3,3,9,5,3,1,0,4,5,4,0,1,6,0,0,1,0
3,Niger Tornadoes,home,2 - 1,W,65,45,174,28,7,7,0,0,0,0,0,0,1,3,7,3,1,19,0,15,6,4,0,0,2,1,2,3,3,1,1,2,0,0,0,5,0,4,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,10,2,34,15,0,20,0,1,14,1,4,0,0,0,0,3,0,1,4,0,0,1,0
4,Remo Stars,away,2 - 1,L,48,36,78,9,5,5,0,0,1,2,0,0,0,1,12,9,3,32,0,18,0,5,21,0,1,2,1,2,5,4,1,4,0,1,0,3,0,9,5,0,0,0,0,0,0,0,3,0,0,0,0,0,1,3,13,0,23,19,1,11,3,2,15,13,5,3,0,1,3,6,0,2,3,0,0,1,0
5,Doma United,home,0 - 0,D,71,38,205,11,11,11,0,0,0,4,0,0,0,1,6,11,9,19,0,25,1,5,1,0,0,2,6,2,3,3,0,4,0,1,1,12,0,5,1,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,10,3,26,10,0,7,0,1,11,6,7,2,0,1,3,5,0,0,8,0,0,1,0
6,Akwa United,away,0 - 0,D,47,43,145,23,11,11,0,0,0,0,0,0,1,0,4,4,4,23,0,14,4,4,0,0,0,0,1,0,2,2,0,1,0,1,0,3,0,6,3,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,17,3,54,16,0,15,3,2,14,6,1,1,0,0,2,6,0,0,2,0,0,2,0
7,Gombe United,home,2 - 0,W,72,34,155,10,13,13,0,0,0,0,0,0,0,2,12,8,12,20,0,17,4,4,0,0,2,1,2,6,5,3,2,9,0,1,0,5,0,8,4,0,0,0,0,0,0,0,3,0,1,0,0,0,1,0,20,5,34,15,1,11,1,0,18,14,1,1,0,2,6,5,0,0,3,0,0,2,0
8,Heartland FC,home,3 - 2,W,39,18,227,23,16,16,0,0,0,0,0,1,1,0,15,1,9,11,0,24,0,0,0,0,3,0,5,5,0,2,0,2,0,2,1,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2,35,19,0,6,2,0,4,5,0,2,0,0,1,1,0,2,2,0,0,3,0
9,Gateway United,home,1 - 0,W,56,37,168,21,7,7,0,0,0,0,0,0,0,1,4,6,4,18,1,12,2,5,0,0,1,1,2,3,1,5,0,4,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,2,37,15,0,11,0,0,8,0,1,1,0,0,0,3,0,0,0,0,0,0,0


In [358]:
full_stats.to_csv("sheets/match_data.csv", index=False)

In [357]:
driver.close()